In [1]:
%env CUDA_VISIBLE_DEVICES=9

from tensorflow.python.client import device_lib
print (device_lib.list_local_devices())

env: CUDA_VISIBLE_DEVICES=9
[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13732871715516252033
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 10978911847
locality {
  bus_id: 1
}
incarnation: 10304481651797778960
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:0f:00.0"
]


In [2]:
# from NLPutils.models.dilatedconv import DilatedConv


In [3]:
# from NLPutils.trainers.dconv_trainer import DConvTrainer


In [4]:
train = '/data/datasets/ner/eng.train'
valid = '/data/datasets/ner/eng.testa'
test = '/data/datasets/ner/eng.testb'
word_embed_loc = '/data/embeddings/GoogleNews-vectors-negative300.bin'
valsplit=0.15

In [5]:
from src.util.conll_util import *
from src.embeddings import Word2VecModel, RandomInitVecModel

maxs, maxw, vocab_ch, vocab_word = conllBuildVocab([train, valid,
                                                    test])

# Vocab LUTs
word_vocab = None
char_vocab = None

word_vec = Word2VecModel(word_embed_loc, vocab_word, 0.25)
word_vocab = word_vec.vocab

# if FLAGS.charsz != FLAGS.wsz and FLAGS.cbow is True:
#     print('Warning, you have opted for CBOW char embeddings, but have provided differing sizes for char embedding depth and word depth.  This is not possible, forcing char embedding depth to be word depth ' + FLAGS.wsz)
#     FLAGS.charsz = FLAGS.wsz

char_vec = RandomInitVecModel(16, vocab_ch, 0.25)
char_vocab = char_vec.vocab

f2i = {"<PAD>":0}

ts, f2i, _ = conllSentsToIndices(train, word_vocab, char_vocab, maxs, maxw, f2i, 3)
print(len(ts))
print('Loaded  training data')

if valid is not None:
    print('Using provided validation data')
    vs, f2i,_ = conllSentsToIndices(valid, word_vocab, char_vocab, maxs, maxw, f2i, 3)
else:
    ts, vs = validSplit(ts, valsplit)
    print('Created validation split')


es, f2i,txts = conllSentsToIndices(test, word_vocab, char_vocab, maxs, maxw, f2i, 3)
print('Loaded test data')


14986
Loaded  training data
Using provided validation data
Loaded test data


In [6]:
print('word embed weights shape: ', word_vec.weights.shape)
print( 'max sentence len', maxs)

word embed weights shape:  (30291, 300)
max sentence len 124


In [7]:
import tensorflow as tf

%load_ext autoreload
%autoreload 2

from src.models.dconv3 import DConv
from src.trainers.dconv3 import DConvTrainer

trainer = DConvTrainer(None, DConv, 'conll-ner-dconv3-9')
trainer.train('ner', ts, f2i, vs, es,
             char_vec,
             word_vec,
             'eval',
              batchsz=2**7,
              optim='adam',
              eta=0.0005,
              epochs=500,
              dropout=0.65,
              patience=50,
              cfiltsz='1,3,5,7',
             maxlen=maxs,
             maxw=maxw,
              num_filt=300,
             num_layers=3,
             num_iterations=2,
             fscore=1,
              viz=1,
             crf=True)

#1 -- 86.55 dropout at .45 and 300 filters. stopped at epoc 24.
#2 -- 86.7 dropout at .35, 350 filters.
#3 -- added dilation 1 conv at end of block. 86.28
#4 -- 3 layers, 3 iters. got much worse. peeked at 91.8 in training, started dropping.
#5 -- layers in block should be 1,2,1 from strubell's code. lowered dropout to .15 -- layers would have been 1,1,1.
#6 -- clipped gradients'
#7 -- 3 layers, 2 iterations.
#8 -- 2**6 batch size, to reflect strubell's config. hit 91.9 in validation, 84.6 in test.
#9 -- upped dropout to .35 from .15. 86.49
#10 -- drop to .65.
#11 -- removed a layer from the block. from 3 (block's dilation should be 1,2,1) to 2, (dilation to 1,1).
#   -- 86.57
#12 -- increasing iterations. blocks have 3 layers again.
#13 -- lowered blocks to 2 layers. decreased eta to .00005. 86.82
#14 -- lowered adam's epsilon from 1e-8 to 1e-6. eta to .0005 from .00005, batch size to 2**7 from 2**6. 86.46

#strubell's dconv3
#1 -- 83.6 F1. wtf is going on.
#2 -- batch size up to 2**7. seems misconfigured with two dropouts..
#3 -- she has middle dropout and hidden dropout in the same place. middle drop is not used, though. i missed this. 87.09
#4  -- added word dropout straight to the word vectors. dropout == .85.  87.54
#   -- I forgot to account for dropout keep in evaluation....
#5 -- setting dropout keep to 1 in evaluation using pkeep and word_keep.  88.12
#6 -- adding in gaussian noise to gradients according to https://arxiv.org/pdf/1511.06807.pdf --
#  -- did flat normal noise, no scaling over t. 88.80. dropout at .65
#7 -- increased noise variance to 0.01 from 0.001. 88.43
#8 -- reverted noise variance to 0.001. changing dropout. my dropout is 1-dropout. it needs to be .15. 88.38
#9 -- dropout back to 0.65. using dropout loss from the paper. in branch refactor.
#  -- creats 11 Sentence level loss functions?? 

/src/src/trainers/dconv3.py:55: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(length == len(guess[b]), "lengths differ: length-- {}, len(guess[b])-- {} ".format(length, len(guess[b])))


block_sore length 2
block_score_no_dropout length 2
block_score length after anothe fwd 2
(?, 124, 9)
crf=True, creating SLL
(?, 124, 9)
(?,)
(?, 124)
(?, 124, 9)
crf=True, creating SLL
(?, 124, 9)
(?,)
(?, 124)


/opt/conda/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Setting up word embedding visualization
Writing metadata
Training epoch 1.
Train (Loss 0.0722) (99.862 sec)
Validation (F1 = 0.7551) (Acc 49185/51409 = 0.9567) (5.513 sec)
Highest dev F1 achieved yet -- writing model
Training epoch 2.
	(last improvement @ 1)
Train (Loss 0.0400) (89.025 sec)
Validation (F1 = 0.8431) (Acc 50101/51409 = 0.9746) (4.809 sec)
Highest dev F1 achieved yet -- writing model
Training epoch 3.
	(last improvement @ 2)
Train (Loss 0.0305) (89.775 sec)
Validation (F1 = 0.8908) (Acc 50492/51409 = 0.9822) (4.899 sec)
Highest dev F1 achieved yet -- writing model
Training epoch 4.
	(last improvement @ 3)
Train (Loss 0.0263) (90.332 sec)
Validation (F1 = 0.8968) (Acc 50534/51409 = 0.9830) (4.992 sec)
Highest dev F1 achieved yet -- writing model
Training epoch 5.
	(last improvement @ 4)
Train (Loss 0.0237) (90.184 sec)
Validation (F1 = 0.9063) (Acc 50631/51409 = 0.9849) (4.527 sec)
Highest dev F1 achieved yet -- writing model
Training epoch 6.
	(last improvement @ 5)
Train

Train (Loss 0.0085) (89.119 sec)
Validation (F1 = 0.9327) (Acc 50823/51409 = 0.9886) (4.944 sec)
Training epoch 54.
	(last improvement @ 39)
Train (Loss 0.0084) (88.968 sec)
Validation (F1 = 0.9311) (Acc 50811/51409 = 0.9884) (4.216 sec)
Training epoch 55.
	(last improvement @ 39)
Train (Loss 0.0084) (87.817 sec)
Validation (F1 = 0.9344) (Acc 50840/51409 = 0.9889) (4.915 sec)
Training epoch 56.
	(last improvement @ 39)
Train (Loss 0.0083) (89.729 sec)
Validation (F1 = 0.9354) (Acc 50854/51409 = 0.9892) (5.203 sec)
Training epoch 57.
	(last improvement @ 39)
Train (Loss 0.0084) (88.428 sec)
Validation (F1 = 0.9337) (Acc 50841/51409 = 0.9890) (5.112 sec)
Training epoch 58.
	(last improvement @ 39)
Train (Loss 0.0083) (89.788 sec)
Validation (F1 = 0.9335) (Acc 50833/51409 = 0.9888) (4.663 sec)
Training epoch 59.
	(last improvement @ 39)
Train (Loss 0.0082) (89.238 sec)
Validation (F1 = 0.9362) (Acc 50841/51409 = 0.9890) (5.024 sec)
Training epoch 60.
	(last improvement @ 39)
Train (Loss 0

Training epoch 111.
	(last improvement @ 79)
Train (Loss 0.0074) (89.475 sec)
Validation (F1 = 0.9315) (Acc 50825/51409 = 0.9886) (4.469 sec)
Training epoch 112.
	(last improvement @ 79)
Train (Loss 0.0073) (89.173 sec)
Validation (F1 = 0.9332) (Acc 50825/51409 = 0.9886) (5.095 sec)
Training epoch 113.
	(last improvement @ 79)
Train (Loss 0.0073) (89.283 sec)
Validation (F1 = 0.9322) (Acc 50823/51409 = 0.9886) (5.020 sec)
Training epoch 114.
	(last improvement @ 79)
Train (Loss 0.0073) (89.298 sec)
Validation (F1 = 0.9330) (Acc 50828/51409 = 0.9887) (4.670 sec)
Training epoch 115.
	(last improvement @ 79)
Train (Loss 0.0074) (90.270 sec)
Validation (F1 = 0.9340) (Acc 50823/51409 = 0.9886) (5.036 sec)
Training epoch 116.
	(last improvement @ 79)
Train (Loss 0.0073) (88.759 sec)
Validation (F1 = 0.9327) (Acc 50828/51409 = 0.9887) (5.237 sec)
Training epoch 117.
	(last improvement @ 79)
Train (Loss 0.0073) (90.406 sec)
Validation (F1 = 0.9351) (Acc 50849/51409 = 0.9891) (4.715 sec)
Traini